# FX Market Making Analytics Portfolio

**Author**: Mame Faty Lo  
**Purpose**: Showcase FX market analysis, data cleaning, visualization, and automation for major currency pairs relevant to Swissquote.  

**Objectives**:
- Load and clean FX market data.
- Analyse **mid-price trends**, **bid-ask spreads**, **returns**, and **volatility**. 
- Simulate **market making strategies** and liquidity provision.
- Generate **interactive charts and static visualisations** to support insights.

**Notes**:
- Data can be loaded from bundled sample CSVs or a snapshot pickle for convenience.
- Volume, bid, and ask columns are included to mimic real FX market data.
- All analysis is reproducible and ready for extension to live data sources.

In [2]:
# Cell 1: Imports
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Set default Plotly template and size
pio.templates.default = "plotly_dark"
px.defaults.width = 900
px.defaults.height = 520

# Import our custom scripts
from src import downloader_fx
from src import data_loader
from src import data_cleaner
print("[INFO] Scripts imported successfully")

[INFO] Scripts imported successfully


# Download FX Data (1-Minute Bars)

We download 1-minute FX data from **HistData.com** for selected currency pairs.  

- Data will be saved as **one CSV per pair** in `data/raw/fx_full/`.  
- Columns include: `DateTime`, `Open`, `High`, `Low`, `Close`, `Volume`.  
- If the CSVs already exist, this step will skip downloading to save time.  
- We fetch **3 months of historical data** as a snapshot for analysis.


In [4]:
# Step 1: Download FX data using our custom downloader

from datetime import date

# Pairs and date range
fx_pairs = ["EURUSD", "USDCHF", "GBPUSD"]
start_date = date(2025, 8, 1)
end_date = date(2025, 10, 28)  # adjust as needed

# Download each pair
for pair in fx_pairs:
    downloader_fx.download_and_combine_fx(pair, start_date, end_date, save_path="data/raw")

[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/01/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/02/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/03/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/04/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/05/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/06/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/07/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/08/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/09/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/10/1min.csv not found (status 403)
[WARN] https://www.dukascopy.com/datafeed/EURUSD/2025/08/11/1min.csv not found (

KeyboardInterrupt: 

In [12]:
# -------------------------
# FX Data Loader
# -------------------------
def load_fx_data(pairs, raw_path=raw_path, snapshot_path=snapshots_path):
    """
    Load FX data for multiple currency pairs.

    Priority:
    1) Load raw CSVs from raw_path if available.
    2) Else load minimal bundled snapshot (pickle).

    Parameters
    ----------
    pairs : list of str
        e.g., ["EUR_USD", "USD_CHF"]
    raw_path : Path
        Folder containing raw CSV files
    snapshot_path : Path
        Folder containing bundled snapshot pickle

    Returns
    -------
    fx_data : dict
        {pair_name: DataFrame}
    """
    fx_data = {}

    for pair in pairs:
        csv_file = raw_path / f"{pair}.csv"
        if csv_file.exists():
            df = pd.read_csv(csv_file)
            print(f"[INFO] Loaded {pair} from CSV: {len(df)} rows")
        else:
            # fallback to snapshot pickle
            snapshot_file = snapshot_path / f"{pair}_snapshot.pkl"
            if snapshot_file.exists():
                df = pd.read_pickle(snapshot_file)
                print(f"[INFO] Loaded {pair} from bundled snapshot: {len(df)} rows")
            else:
                print(f"[WARNING] No CSV or snapshot found for {pair}, skipping")
                continue
        # Convert date column if present
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        # Ensure numeric columns
        for col in ["Price", "Open", "High", "Low", "Volume"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
        fx_data[pair] = df

    return fx_data

# -------------------------
# Example usage
# -------------------------
pairs = ["EUR_USD", "USD_CHF", "GBP_USD"]
fx_data = load_fx_data(pairs)


[WARNING] No CSV or snapshot found for EUR_USD, skipping
[WARNING] No CSV or snapshot found for USD_CHF, skipping
[WARNING] No CSV or snapshot found for GBP_USD, skipping


## Load API Keys from Environment Variables

We keep API credentials separate from the notebook for security and reproducibility. This cell loads your OANDA (FX) and Binance (Digital Assets) API keys from the `.env` file located in the `env/` folder at the project root. Keys will be stored in variables for subsequent data fetching.

In [13]:
from datetime import date
download_and_combine_fx("EURUSD", date(2025, 8, 1), date(2025, 10, 28))

NameError: name 'download_and_combine_fx' is not defined

In [ ]:
# Info for each FX pair
for pair_name, df in cleaned_fx_data.items():
    print(f"=== {pair_name} ===")
    df.info()
    print("\n")

# Descriptive stats (excluding 'Date')
for pair_name, df in cleaned_fx_data.items():
    print(f"=== {pair_name} ===")
    display(df[numeric_cols].describe())

## FX Market Analysis

### Basic Descriptive Statistics

Quick descriptive statistics for numeric columns (`Open`, `High`, `Low`, `Price`, `Change %`) across all FX pairs. This helps inspect ranges, averages, and variability.

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
print("Missing dates:", df["Date"].isna().sum())

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        print(f"Missing in {col}:", df[col].isna().sum())

### Price Trends (Normalised)

**Purpose:** Compare relative price movements across FX pairs that have very different scales.

**Method:** We normalise each pair by its price at the first available date:

$$
\text{Normalised Price}_t = \frac{\text{Price}_t}{\text{Price}_{t_0}}
$$

Where:
- $t_0$ is the first available date for the FX pair.
- $\text{Price}_t$ is the price at time $t$.

In [14]:
# Concatenate all cleaned pairs into a single dataframe
plot_df = pd.concat(
    (df.assign(Pair=pair) for pair, df in cleaned_fx_data.items()),
    ignore_index=True
)

# Ensure 'Date' column is datetime
plot_df["Date"] = pd.to_datetime(plot_df["Date"], errors="coerce")

# 3) Convert numeric columns to float
numeric_cols_existing = [c for c in numeric_cols if c in plot_df.columns]
plot_df[numeric_cols_existing] = plot_df[numeric_cols_existing].apply(pd.to_numeric, errors="coerce")

# 4) Drop rows with missing Date, Price, or Pair, then sort
plot_df = plot_df.dropna(subset=["Date", "Price", "Pair"]).sort_values(["Pair", "Date"]).reset_index(drop=True)

# 5) Compute normalised price per pair (base = first available price)
plot_df["Normalised Price"] = plot_df.groupby("Pair")["Price"].transform(lambda x: x / x.iloc[0])

# 6) Quick check: number of points per pair & preview
print("Points per pair:\n", plot_df["Pair"].value_counts(), "\n")
display(plot_df.loc[:, ["Pair", "Date", "Price", "Normalised Price"]].head(8))

# 7) Interactive Plotly line chart
fig = px.line(
    plot_df,
    x="Date",
    y="Normalised Price",
    color="Pair",
    title="Normalised FX Price Trends (base = first available price)",
    labels={
        "Normalised Price": "Normalised Price (base = 1)",
        "Date": "Date",
        "Pair": "Currency Pair"
    },
)

fig.update_layout(
    template="plotly_dark",
    legend_title_text="Currency Pair",
    width=960,
    height=520,
    hovermode="x unified",
)

fig.show()

NameError: name 'cleaned_fx_data' is not defined

- All pairs start at 1, enabling direct visual comparison of relative price movements.
- Hover over lines to see exact normalised values and dates.

## Daily Returns and Volatility

**Purpose**:  
Assess the day-to-day price changes and the risk (volatility) associated with each FX pair. This helps understand market behaviour and identify periods of high or low activity.

**Method**:  

1. **Daily Returns**: The percentage change in FX price from one day to the next:  

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} \times 100
$$

Where:  
- \(R_t\) = daily return at day \(t\)  
- \(P_t\) = FX price at day \(t\)  
- \(P_{t-1}\) = FX price at day \(t-1\)  

2. **Volatility**: Standard deviation of daily returns over a period \(n\):  

$$
\text{Volatility}_t = \sqrt{\frac{1}{n-1} \sum_{i=0}^{n-1} (R_{t-i} - \bar{R})^2}
$$

Where:  
- \(R_{t-i}\) = daily return on day \(t-i\)  
- \(\bar{R}\) = mean daily return over \(n\) days  

**Highlight**:  
- Use an interactive Plotly line chart to visualise returns over time.  
- Volatility can be plotted as a rolling standard deviation to show periods of higher risk.  
- Colour-code FX pairs for clarity and hover labels for exact values.

## Daily Returns and Volatility

**Purpose**:  
Assess the day-to-day price changes and the risk (volatility) associated with each FX pair. This helps understand market behaviour and identify periods of high or low activity.

**Method**:  

1. **Daily Returns**: The percentage change in FX price from one day to the next:  

$$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} \times 100
$$

Where:  
- \(R_t\) = daily return at day \(t\)  
- \(P_t\) = FX price at day \(t\)  
- \(P_{t-1}\) = FX price at day \(t-1\)  

2. **Volatility**: Standard deviation of daily returns over a period \(n\):  

$$
\text{Volatility}_t = \sqrt{\frac{1}{n-1} \sum_{i=0}^{n-1} (R_{t-i} - \bar{R})^2}
$$

Where:  
- \(R_{t-i}\) = daily return on day \(t-i\)  
- \(\bar{R}\) = mean daily return over \(n\) days  

**Highlight**:  
- Use an interactive Plotly line chart to visualise returns over time.  
- Volatility can be plotted as a rolling standard deviation to show periods of higher risk.  
- Colour-code FX pairs for clarity and hover labels for exact values.

In [ ]:
# --- Compute daily returns and rolling volatility ---

returns_df = pd.DataFrame()

for pair, df in cleaned_fx_data.items():
    temp = df.copy()
    temp["Daily Return %"] = temp["Price"].pct_change() * 100
    temp["Volatility (5-day rolling %)"] = temp["Daily Return %"].rolling(window=5).std()
    temp["Pair"] = pair
    returns_df = pd.concat([returns_df, temp], ignore_index=True)

# Drop first row of each pair (NaN return)
returns_df.dropna(subset=["Daily Return %"], inplace=True)

# Interactive Plotly line chart for daily returns
fig_returns = px.line(
    returns_df,
    x="Date",
    y="Daily Return %",
    color="Pair",
    title="Daily Returns for Major FX Pairs",
    labels={"Daily Return %": "Daily Return (%)", "Date": "Date", "Pair": "Currency Pair"}
)
fig_returns.update_layout(template="plotly_dark", hovermode="x unified")
fig_returns.show()

# Optional: Plot rolling volatility
fig_vol = px.line(
    returns_df,
    x="Date",
    y="Volatility (5-day rolling %)",
    color="Pair",
    title="Rolling 5-Day Volatility of FX Pairs",
    labels={"Volatility (5-day rolling %)": "Volatility (%)", "Date": "Date", "Pair": "Currency Pair"}
)
fig_vol.update_layout(template="plotly_dark", hovermode="x unified")
fig_vol.show()